In [1]:
!pip3 install tensorflow 
!pip3 install yahoo_fin 
!pip3 install requests_html

     |████████████████████████████████| 153kB 7.0MB/s 
     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 71kB 4.9MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=540731078ea69d5eb18e04bee991535d6b24bdf9db8c1d62e6f071a38e2cee8d
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.18.0-cp36-none-any.whl size=24133 sha256=6ddcdf2e85f7489d3dae281235c0da0c474701ebbe3c028c7cc822bba97a239b
  Stored in directory: /root/.cache/pip/wheels/2a/53/09/869ca5781ede342254ffac09ca99461b008c3e5f8dd079b0c0
Successfully built fake-useragent parse
ERROR: pyppeteer 0.2.2 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from yahoo_fin import stock_info as si
from collections import deque
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime, timedelta
import csv
import pandas_datareader.data as web
import pandas_datareader as pdr
from pandas_datareader import data, wb
import time
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like #For solving import pandas_datareader issue
import os
import random
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import urlopen

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/NIFTY_50'             # get the NIFTY50 companies from wikipedia
html1=urlopen(wiki_url)
soup = BeautifulSoup(html1, 'html.parser')                # Extracting names from  HTML page
nse50data1=""          

table = soup.find_all('table', class_="wikitable")[0]         #retrieve all descendants that match  filter

for record in table.findAll('tr'):
    nse50data=""
    for data in record.findAll('td'):
        nse50data=nse50data+","+data.text  
    nse50data1=nse50data1+"\n"+nse50data[1:]

print(nse50data1)
f= open("nse50.txt","w+")

f.write(nse50data1)
f.close()
headers = []
for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
rows = []
for tr in table.find_all("tr")[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
           
            ths = tr.find_all("th")
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
d=pd.DataFrame(rows, columns=headers).to_csv(f"table.csv")



Adani Ports,ADANIPORTS.NS,Infrastructure

Asian Paints,ASIANPAINT.NS,Consumer Goods

Axis Bank,AXISBANK.NS,Banking

Bajaj Auto,BAJAJ-AUTO.NS,Automobile

Bajaj Finance,BAJFINANCE.NS,Financial Services

Bajaj Finserv,BAJAJFINSV.NS,Financial Services

Bharti Airtel,BHARTIARTL.NS,Telecommunication

Bharti Infratel,INFRATEL.NS,Telecommunication

Bharat Petroleum,BPCL.NS,Energy - Oil & Gas

Britannia Industries,BRITANNIA.NS,Consumer Goods

Cipla,CIPLA.NS,Pharmaceuticals

Coal India,COALINDIA.NS,Energy & Mining

Dr. Reddy's Laboratories,DRREDDY.NS,Pharmaceuticals

Eicher Motors,EICHERMOT.NS,Automobile

GAIL,GAIL.NS,Energy - Oil & Gas

Grasim Industries,GRASIM.NS,Cement

HCL Technologies,HCLTECH.NS,Information Technology

HDFC,HDFC.NS,Financial Services

HDFC Bank,HDFCBANK.NS,Banking

HDFC Life,HDFCLIFE.NS,Insurance

Hero MotoCorp,HEROMOTOCO.NS,Automobile

Hindalco Industries,HINDALCO.NS,Metals

Hindustan Unilever,HINDUNILVR.NS,Consumer Goods

ICICI Bank,ICICIBANK.NS,Banking

IndusInd Bank,I

In [4]:
# Import list of stock names from NSE website
#this is actually nse bhavcopy 
with requests.Session() as s:
    download = s.get('https://www1.nseindia.com/products/content/sec_bhavdata_full.csv')
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    df = pd.DataFrame(list(cr))
    
#View the top rows
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,SYMBOL,SERIES,DATE1,PREV_CLOSE,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,LAST_PRICE,CLOSE_PRICE,AVG_PRICE,TTL_TRD_QNTY,TURNOVER_LACS,NO_OF_TRADES,DELIV_QTY,DELIV_PER
1,20MICRONS,EQ,25-Sep-2020,26.20,26.80,30.20,26.80,29.50,28.45,28.10,54368,15.28,460,31523,57.98
2,21STCENMGM,EQ,25-Sep-2020,10.20,10.00,10.30,10.00,10.30,10.15,10.09,3282,0.33,50,2536,77.27
3,3IINFOTECH,EQ,25-Sep-2020,3.05,3.05,3.15,2.90,2.95,2.90,2.93,9668130,283.71,2745,4584932,47.42
4,3MINDIA,EQ,25-Sep-2020,18367.45,18501.00,18657.00,18150.00,18190.00,18204.90,18244.86,5482,1000.18,3620,3534,64.47


In [5]:
np.random.seed(314)
tf.random.set_seed(314)
random.seed(314)

In [6]:

print("(CHOOSE THE TICKER FROM DATAFRAME MENTIONED ABOVE)")
print("\nEnter the ticker")
stock_in=input()              #User input for the ticker
def load_data(ticker=stock_in, n_steps=50, scale=True, shuffle=True, lookup_step=1, 
                test_size=0.2, feature_columns=['adjclose', 'volume', 'open', 'high', 'low']):
    """
    Loads data from Yahoo Finance source, as well as scaling, shuffling, normalizing and splitting.
    Params:
        ticker (str/pd.DataFrame): the ticker you want to load, examples include AAPL, TESL, etc. You can add the ticker from list of niftyfifty companies shown above.
        n_steps (int): the historical sequence length (i.e window size) used to predict, default is 50
        scale (bool): whether to scale prices from 0 to 1, default is True
        shuffle (bool): whether to shuffle the data, default is True
        lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
        test_size (float): ratio for test data, default is 0.2 (20% testing data)
        feature_columns (list): the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
    """
    # see if ticker is already a loaded stock from yahoo finance
    if isinstance(ticker, str):
        # load it from yahoo_fin library
        df = si.get_data(ticker)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    # this will contain all the elements we want to return from this function
    result = {}
    # we will also return the original dataframe itself
    result['df'] = df.copy()
    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."
    if scale:
        column_scaler = {}
        # scale the data (prices) from 0 to 1
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler
        # add the MinMaxScaler instances to the result returned
        result["column_scaler"] = column_scaler
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['adjclose'].shift(-lookup_step)
    # last `lookup_step` columns contains NaN in future column
    # get them before droping NaNs
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    # drop NaNs
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    for entry, target in zip(df[feature_columns].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
    # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
    # for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 60 (that is 50+10) length
    # this last_sequence will be used to predict future stock prices not available in the dataset
    last_sequence = list(sequences) + list(last_sequence)
    last_sequence = np.array(last_sequence)
    # add to result
    result['last_sequence'] = last_sequence
    # construct the X's and y's
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    # reshape X to fit the neural network
    X = X.reshape((X.shape[0], X.shape[2], X.shape[1]))
    # split the dataset
    result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, 
                                                                               test_size=test_size, shuffle=shuffle)
    # return the result
    return result

(CHOOSE THE TICKER FROM DATAFRAME MENTIONED ABOVE)

Enter the ticker
SBIN.NS
